In [14]:
import cv2
import os
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.svm import LinearSVC, SVC
from sklearn.externals import joblib
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# image function

In [2]:
def imlist(path): # list the images in a folder
    """
    The function imlist returns all the names of the files in 
    the directory path supplied as argument to the function.
    """
    return [os.path.join(path, f) for f in os.listdir(path)]

# bag-of-word (BOW) function

In [3]:
# Create feature extraction and keypoint detector objects
def BOW(image_paths, sift):
    des_list = []     # save descriptor list
    for image_path in image_paths:
        im = cv2.imread(image_path)
        gray= cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        (kps, des) = sift.detectAndCompute(gray, None)
        des_list.append((image_path, des))  
    return des_list

# histogram of BOW function

In [4]:
def hist_bow(des_list, image_paths):
    # Stack all the descriptors vertically in a numpy array
    descriptors = des_list[0][1]
    for image_path, descriptor in des_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))  
    
    # Perform k-means clustering
    k = 50
    voc, variance = kmeans(descriptors, k, 1) 
    # Calculate the histogram of features
    im_features = np.zeros((len(image_paths), k), "float32")
    for i in range(len(image_paths)):
        words, distance = vq(des_list[i][1],voc)
        for w in words:
            im_features[i][w] += 1
    return im_features

# Load train data and test data

In [5]:
# Get the path of the dataset
path = r'traindata'
names = os.listdir(path)

In [6]:
# Get all the path to the images and save them in a list; save the image label to a list
image_paths = []
image_classes = []
class_id = 0
for name in names:
    dir = os.path.join(path, name)
    class_path = imlist(dir)
    image_paths+=class_path
    image_classes+=[class_id]*len(class_path)
    class_id+=1

In [7]:
# initiate the sift algorithm
sift = cv2.xfeatures2d.SIFT_create()

# train data
descriptor = BOW(image_paths, sift)
im_features = hist_bow(descriptor, image_paths)

# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
# Calculating the number of occurrences
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')
# Giving weight to one that occurs more frequently
im_features = im_features*idf

# Scaling the words
stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)

In [15]:
#### 5-folder cross validatiaon ####
# accuracy
clf = SVC()
scores = cross_val_score(clf, im_features, image_classes, cv=5)
print("the average accuracy is ", np.mean(scores))

# confusion matrix
predict = cross_val_predict(clf, im_features, image_classes, cv=5)
confusion_matrix(image_classes, predict)

the average accuracy is  0.47750000000000004


array([[46, 11,  5,  5,  3,  1, 26,  3],
       [ 0, 89,  0,  0,  3,  1,  2,  5],
       [ 8, 23, 28,  4,  0,  4, 29,  4],
       [ 5,  5,  3, 50, 18,  2,  7, 10],
       [ 9, 10,  1, 12, 51,  0,  8,  9],
       [ 6, 18, 13, 11,  4, 27, 19,  2],
       [13, 22,  8,  6,  8,  3, 36,  4],
       [ 1, 12,  0,  8, 18,  0,  6, 55]])